# Laboratorio 5: Programación Lineal Mixta (MA4702, otoño 2023)

Fecha: 28 de junio, 2023.<br/>
Profesor: Marcos Kiwi.<br/>
Auxiliares: Benjamín Jauregui y Cristián Palma.<br/>

## Preliminares

Este laboratorio consta de 2 etapas. Una Tarea Introductoria (TI) y una etapa de Trabajo Presencial (TP). La nota global del laboratorio será 40 % (TI) + 60 % (TP).

La tarea introductoria es de carácter estrictamente individual. La etapa presencial es de carácter grupal. 


## Laboratorio 5 Trabajo Presencial (TP)

## Instrucciones.

En la parte presencial del laboratorio, realice las siguientes actividades:

* Complete la celda “Datos grupo”.
* Descargue los archivos y funciones auxiliares de UCursos (todos deben quedar en el mismo directorio en que está almacenado este archivo).
* Lea el enunciado y complete los ejercicios 1, 2 y 3.
* Envíe el archivo .ipynb por el módulo de tareas de UCursos en la TAREA: Laboratorio 4 presencial. 
* Plazo de entrega: **miércoles 28 de junio a las 18:00** (se descontará 0.5 pts. por cada 15 min o fracción de atraso).

<div class="alert alert-block alert-warning">
    Advertencia: Si en el archivo que entregue alguna de las celdas arroja un mensaje de error al ejecutar, o el código entra en un loop infinito y no retorna con ninguna salida, Ud. deberá agregar una nota al respecto, explicando cuál cree que es el problema. Si no lo hace, se le penalizará adicionalmente.               
</div>


## Datos personales

<div class="alert alert-block alert-success">

- Número o identificador del Grupo: 
    
- Integrante 1: XXX

- Integrante 2: YYY     
</div>


## Introducción: Programación estocástica y descomposición de Benders.

El objetivo de esta parte del laboratorio es estudiar un tipo de problema lineal estocástico susceptible de ser abordado a través del método de descomposición de Benders. Este tipo de problemas son frecuentes en contextos como el diseño de redes de generación de electricidad, localización de plantas industriales, etc.

Específicamente, vamos a considerar el siguiente tipo de programa estocástico lineal mixto:

$$
\begin{array}{rlll}
(PL_0):\ z & =\min_{x, y} \alpha^Tx + \mathbb{E}\big(\beta^T y^{\sigma}\big) \\
\text{s.a.} & Ax \geq \gamma, \\
& A^\sigma x+By^\sigma \geq \gamma^\sigma, & \forall \sigma\in\Sigma, \\
& y^\sigma \geq \vec{0}, & \forall \sigma\in\Sigma, \\
& x\geq\vec{0}, 
\end{array}
$$

donde la esperanza está tomada sobre los posibles escenarios $\sigma\in\Sigma$. 


Observar que podemos reformular $(PL_0)$ de la siguiente forma:

$$
\begin{array}{rlll}
z &  = \min_{x} \alpha^Tx + \sum_{\sigma\in\Sigma}p_\sigma z_{\sigma}(x) \\
\text{s.a.} & Ax \geq \gamma, \\
& x\geq\vec{0}, 
\end{array} 
$$

donde $p_\sigma$ es la probabilidad que ocurra el escenario $\sigma$ (sin pérdida de generalidad, asumimos que $p_\sigma>0$ para todo $\sigma\in\Sigma$) y

$$
\begin{array}{rlll}
z^\sigma(x) & = \min_{y} \beta^Ty \\
\text{s.a.} & B y\geq \gamma^\sigma-A^\sigma x, \\
& y\geq\vec{0}.
\end{array}
$$

Dualizando, obtenemos:

$$
\begin{array}{rlll}
(SubP^{\sigma}_x):\ z^\sigma(x) & = \max_{w} w^T(\gamma^\sigma-A^\sigma x) \\
\text{s.a.} & w^TB \leq \beta, \\
& w\geq\vec{0}.
\end{array}
$$

Definimos $\mathcal{P}=\{w \mid w^TB \leq \beta, w\geq\vec{0}\}$. Asumiendo que $\mathcal{P}$ es puntiagudo, por Minkowski-Weyl, sigue que $\mathcal{P}=\mathrm{conv}(\mathcal{V})+\mathrm{cono}(\mathcal{R})$ donde $\mathcal V$ es conjunto finito de puntos extremos y $\mathcal{R}$ es conjunto finito de rayos extremos de $\mathcal{P}$, respectivamente. Observar que mientras el número de variables en $(PL_0)$ es enorme, cada uno de los $(SubP^\sigma_x)$ involucra un número significativamente menor de variables (independiente de la cantidad de escenarios posibles). Además, solo la función objetivo depende del escenario $\sigma$ en $(SubP^\sigma_x)$, mientras que el conjunto factible no.

Sigue que si $z>-\infty$, entonces una reformulación equivalente de $(PL_0)$ es:

$$
\begin{array}{rlll}
(PL):\ z & = \min_{x,\zeta} c^Tx+\sum_{\sigma\in\Sigma}p_\sigma\zeta^\sigma \\
\ \ \text{s.a.}
& v^T(\gamma^\sigma-A^\sigma x) \leq \zeta^\sigma, & \forall v\in\mathcal{V}, \forall\sigma\in\Sigma,\\
& r^T(\gamma^\sigma-A^\sigma x) \leq \vec{0}, & \forall r\in\mathcal{R}, \forall\sigma\in\Sigma,\\
& \zeta\in\mathbb{R}^\Sigma, \\
& x\geq\vec{0}.
\end{array}
$$

La discusión anterior motiva el siguiente procedimiento iterativo para resolver $(PL)$:

> **Descomoposición de Benders.**
> 1. Definir el *master problem* inicial $(MP)$ versión de $(PL)$ donde se han omitido gran parte de sus restricciones (y/o reemplazado por desigualdades válidas).
> 2. Obtener $(x^\star,\zeta^\star)$, $\zeta^\star=(\zeta^\sigma_\star : \sigma\in\Sigma)$ solución óptima de $(MP)$.
> 3. Si para todo $\sigma\in\Sigma$ el programa $(SubP^\sigma_{x^\star})$ es factible y su valor óptimo es a lo más $\zeta^\sigma_\star$, entonces $x^\star$ es solución óptima. Reemplazar en $(PL_0)$, determinar el correspondiente valor de $y^\star$ y **parar**.
> 4. Para todo $\sigma\in\Sigma$ tal que el programa $(SubP^\sigma_{x^\star})$ tiene solución óptima cuyo valor es al menos $\zeta^\sigma_\star$, identificar una s.b.f. $v$ de $(SubP^\sigma_{x^\star})$, agregar la restricción $v^T(b-Ax)\leq \zeta^\sigma$ a la instancia actual de $(MP)$, e ir a 2.
> 5. Para todo $\sigma\in\Sigma$ tal que el programa $(SubP^\sigma_{x^\star})$ es no acotado, identificar un rayo extremo $r$ de $(Sub^\sigma_{x^\star})$ de costo positivo, agregar la restricción $r^T(b-Ax)\leq \vec{0}$ a la instancia actual de $(MP)$, e ir a 2.

**Nota:** La discusión anterior sigue siendo válida aun si $x$ es integral. En este caso, en vez de un programa lineal como $(PL)$ se tiene un programa lineal mixto. El procedimiento descrito puede ser útil para resolver la relajación del programa mixto en cada nodo de un procedimiento de *branch & bound*. Para esto, lo conveniente es agregar las restricciones usando *lazy constraints* como se vio en un laboratorio anterior. 

## Problema de la expansión de la capacidad de energía eléctrica.

Una compañía eléctrica quiere instalar generadores (indexados por elementos de $J$) con diferente costo fijo y operación, con el fin de satisfacer la demanda dentro de su región de servicio. Cada día se divide en periodos de igual duración, indexadas por $i\in I$. Estos corresponden a partes del día durante las cuales la demanda toma un valor base, medio o alto, respectivamente. El costo fijo por unidad de capacidad del generador $j\in J$ se amortiza a lo largo de su vida útil y asciende a $c_j\in\mathbb{R}_+$ por día. El costo operativo del generador $j\in J$ durante la $i$-ésima parte del día es $f_{i,j}\in\mathbb{R}_+$ por unidad. Si la demanda durante la $i$-ésima parte del día no puede ser servida debido a la falta de capacidad, se debe comprar capacidad adicional a un costo de $g_i\in\mathbb{R}_+$ por unidad adicional. Finalmente, se requiere que la capacidad de cada generador $j\in J$ sea al menos $b_j\in\mathbb{R}_+$.

Hay dos fuentes de incertidumbre, a saber, el valor exacto de la demanda durante cada parte $i\in I$ del día, y la disponibilidad $a_j\in\mathbb{R}_+$ del generador $j\in J$. Durante el periodo $i$ la demanda puede tomar uno de $t_i$ valores $d_{i,1}, ..., d_{i,t_i}\in\mathbb{R}_+$, con probabilidad $p_{i,1}, ..., p_{i,t_i}$, respectivamente. La disponibilidad del generador $j$ puede ser $a_{j,1},..., a_{j,k_j}$, con probabilidad $q_{j,1}, ..., q_{j,k_j}$, respectivamente. Sigue que el conjunto de escenarios posibles está dado por: 

$$
\Sigma = \big(\times_{i\in I}[t_i]\big) \times \big(\times_{j\in J} [k_j]\big).  
$$

Si enumeramos todos los posibles eventos, vemos que hay un total de $(\prod_{i\in I}t_i)(\prod_{j\in J}k_j)$ escenarios $\sigma\in\Sigma$ posibles. Denotamos por $d_i^\sigma$ y $a_j^\sigma$ las demandas durante el periodo $i$ del día y las disponibiliaddes del generador $j$, respectivamente, en el escenario $\sigma$. Observar que la probabilidad $p_\sigma$ que ocurra el escenario $\sigma=(\vec{d},\vec{a})$ donde $\vec{d}=(d_{i,h_i} : i\in I)$ y $\vec{a}=(a_{j,\ell_j} : j\in J)$ es 

$$
p_\sigma = \big(\prod_{i\in I}p_{i,h_i}\big)\big(\prod_{j\in J}q_{j,\ell_j}\big).
$$

Introducimos las variables de primera etapa $x_j$ con $j\in J$, que representan la capacidad instalada del generador $j$. También introducimos las variables de segunda etapa $y_{i,j}^{\sigma}$ que denotan los niveles de operación del generador $j$ durante la $i$-ésima parte del día y bajo el escenario $\sigma$. Finalmente,  $\overline{y}_i^{\sigma}$ es la capacidad que se necesita comprar en el escenario $\sigma$, con el fin de satisfacer la demanda insatisfecha durante la $i$-ésima parte del día. Interpretaremos la disponibilidad en el sentido de que el nivel operativo del generador $j$, en el escenario $\sigma$ y cualquier instante de tiempo, es como máximo $a^\sigma_jx_j$. Llegamos entonces a la siguiente formulación:

$$
\begin{array}{rlll}
(PL'):\ & \min \sum_{j\in J} c_j x_j +\mathbb{E}\Big(\sum_{i\in I}\Big(\sum_{j\in J} f_{i,j}y_{i,j}^\sigma+g_i\overline{y}_i^\sigma\Big)\Big) \\
\text{s.a.} & x_j\geq b_j, & \forall j\in J, \\
& a_{j}^\sigma x_j-y_{i,j}^{\sigma} \geq 0, & \forall i\in I, j\in J, \sigma\in\Sigma, \\
& \sum_{j\in J} y_{i,j}^\sigma + \overline{y}_i^\sigma \geq d_i^\sigma, & \forall i\in I, \sigma\in\Sigma, \\
& y_{i,j}^\sigma, \overline{y}_i^\sigma\geq 0, & \forall i\in I, j\in J, \sigma\in\Sigma, \\
& x\geq\vec{0}.
\end{array}
$$

Observar que
$$
\mathbb{E}\Big(\sum_{i\in I}\Big(\sum_{j\in J} (f_{i,j}y_{i,j}^\sigma+g_i\overline{y}_i^\sigma\Big)\Big)
=
\sum_{\sigma\in\Sigma} p_{\sigma}\Big(\sum_{i\in I}\Big(\sum_{j\in J} f_{i,j}y_{i,j}^\sigma+g_i\overline{y}_i^\sigma\Big)\Big).
$$

Por lo tanto, la función de costo en $(PL')$ es lineal. Además, $(PL')$ es un caso particular de $(PL)$ descrito en la sección previa. 


## Ejercicio 1.

En la siguiente celda se proveen los datos de una instancia del problema de expansión de la capacidad de energía eléctrica descrito en la sección anterior, y a través de un procedimiento proveído en el archivo `lab5TP-funcionalidad.jl`, se generan los siguientes 3 arreglos que pueden ser de utilidad en el desarrollo del laboratorio:

* `dmnd[]`: Arreglo 2-dimensional de $|I|\times |\Sigma|$ donde `dmnd[i,s]` representa el valor de $d_i^s$.

* `avlb[]`: Arreglo 2-dimensional de $|J|\times |\Sigma|$ donde `avlb[j,s]` representa el valor de $a_j^s$.

* `prob[]`: Arreglo de dimensión $|\Sigma|$ donde `prob[s]` representa $p_s$.

In [3]:
# No. y conjunto de índices de los periodos de un día.
nI = 3; I = 1:nI;

# No. y conjunto de índices de los generadores.
nJ = 2; J = 1:nJ

# Valores de d_{i,h}: Posibles demandas por periodo del día.
d = Array{Array{Integer}}(undef,nI)
d = [ [35, 42, 28, 22], [43, 55, 32, 30, 60], [54, 66, 39, 40, 20] ] 

# Valores de a_{j,\ell}: Posibles disponibilidades por generador.
a = Array{Array{Integer}}(undef,nJ)
a = [ [10, 2, 0], [5, 10, 0] ]

# Valores de p_{i,h}: Probabilidades de que la demanda sea d_{i,h} en el periodo i del día.
p = Array{Array{Float64}}(undef,nI)
p = [ [0.65, 0.2, 0.1, 0.05], [0.4, 0.4, 0.075, 0.075, 0.05], [0.1, 0.2, 0.3, 0.35, 0.05] ]

# Valores de q_{j,\ell}: Probabilidad que la disponibilidad del generador j sea a_{j,\ell}.
q = Array{Array{Float64}}(undef,nJ)
q = [ [0.7, 0.25, 0.05], [0.7, 0.25, 0.05] ] 

# Capacidad mínima requerida por generador.
b = Vector{Integer}(undef,nI)
b = [ 10, 3 ]

# Costos fijos por generador
c = Vector{Integer}(undef,nJ)
c = [ 3000, 8000 ]

# Costo por unidad de capacidad adicional adquirida por periodo del día.
g = Vector{Integer}(undef,nI)
g = [ 300, 800, 500 ]

# Costo operacional por unidad por periodo del día y generador.
f = Matrix{Float64}(undef,nI,nJ)
f = [100 110; 100 110; 110 120]

#
# Cálculo de los arreglos dmnd[], avlb[], y prob[]. 
#
include("lab5TP-funcionalidad.jl")
dmnd, avlb, prob = genData( a, d )

# Determinación del número de escenarios y el conjunto de índices de los escenarios posibles.
nS = size(prob,1); S = 1:nS;


1.a) (2.0 pts) Complete la siguiente celda de modo de resolver $(PL')$. Determine el óptimo y el valor de $x$ en una solución óptima.

In [1]:
using JuMP, Gurobi
const GUROBI_ENV = Gurobi.Env() 

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-22


Gurobi.Env(Ptr{Nothing} @0x000001ca4e923240, false, 0)

In [3]:

mdl = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV),"OutputFlag" => 0, "Presolve" => 0)) 

@variable(mdl, x[1:nJ] ≥ 0)
@variable(mdl, y[1:nS, 1:nI, 1:nJ] ≥ 0)
@variable(mdl, y_bar[1:nS, 1:nI] ≥ 0)

@constraint(mdl, [j ∈ 1:nJ], x[j] ≥ b[j])
@constraint(mdl, [s ∈ S, i ∈ 1:nI, j ∈ 1:nJ], avlb[j, s]*x[j] - y[s, i, j] ≥ 0)
@constraint(mdl, [s ∈ S, i ∈ 1:nI], sum(y[s, i, 1:end]) + y_bar[s, i] ≥ dmnd[i, s])

@objective(mdl, Min, sum(c .* x) + sum(prob[s]*sum(sum(f[i, j]*y[s, i, j] for j ∈ 1:nJ) + g[i]*y_bar[s, i] for i ∈ 1:nI) for s ∈ S))

optimize!(mdl)

opt_val = objective_value(mdl)
x_val = value.(x)

println( "Valor óptimo: ", opt_val )
println( "x = ", x_val)

Valor óptimo: 71949.92500000038


x = 

[10.0, 3.0]


1.b) (4.0 pts) Vuelva a resolver la instancia de $(PL')$ de la parte 1.a) pero ahora haciendo descomposición de Benders de acuerdo a lo descrito al final de la sección Introducción. Considere el siguiente *master problem* inicial:

$$
\begin{array}{rlll}
& \min_{x,\zeta} c^Tx+\sum_{\sigma\in\Sigma}p_\sigma\zeta^\sigma \\
\ \ \text{s.a.}
& x_j\geq b_j, & \forall j\in J, \\
& \zeta\in\mathbb{R}_+^\Sigma, \\
& x\geq\vec{0}.
\end{array}
$$

Por cada iteración, genere la siguiente salida:

```
======= Iter t =======   # donde t es el número de la iteración
======= Master =======
<status_mstr>            # status del master problem (e.g., OPTIMAL)
<valor_obj_mstr>         # valor objetivo del master problem.
<valor_x>                # valor de x

====== SubProbs ======
     .
     .
     .
<restricciones>          # listado de nuevas restricciones generadas.
     .
     .
     .
```

In [9]:
maxIter = 4

println( "======= Iter 0 ======" ); 
println( "======= Master ======" );

mstr = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV), "OutputFlag" => 0));
@variable( mstr, ζ[1:nS] ≥ 0)
@variable( mstr, x[1:nJ] ≥ 0)
@constraint( mstr,[j ∈ 1:nJ], x[j] ≥ b[j])
@objective( mstr, Min, sum(c.*x) + sum(prob .* ζ))
optimize!( mstr )

println( "===== SubProbs =====" );

subProbs = []
for s ∈ S
    subProb = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(GUROBI_ENV), "OutputFlag" => 0, "Presolve" => 0 ));
    @variable(subProb, w[1:nI, 1:nJ] >= 0)
    @variable(subProb, v[1:nI] ≥ 0)
    @constraint(subProb, [i ∈ 1:nI, j ∈ 1:nJ], sum(v) - w[i, j] ≤ f[i, j])
    @constraint(subProb, [i ∈ 1:nI], sum(v) ≥ g[i])
    @objective(subProb, Max, sum(dmnd[1:end, s].*v) - sum(avlb[j, s]*value(x[j]) * sum(w[i, j] for i in 1:nI) for j in 1:nJ))
    optimize!(subProb);
    push!( subProbs, subProb )
end


for iter in 1:maxIter
    println( "======= Iter $iter ======" );

    prtinln("======= Master ======")
    optimize!( mstr )
    ζ_val = value.(ζ)
    println(MOI.termination_status(mstr))
    println( "Valor óptimo: ", objective_value(mstr) )
    println( "x = ", value.(x) )
    add_ray_restriction = []
    add_vertex_restriction = []
    canbreak = true
    for s ∈ S
        @objective(subProbs[s], Max, sum(dmnd[1:end, s].*subProbs[s][:v]) - sum(avlb[j, s]*value(x[j]) * sum(subProbs[s][:w][i, j] for i in 1:nI) for j in 1:nJ))
        optimize!(subProbs[s])
        if termination_status(subProbs[s]) != MOI.OPTIMAL
            canbreak = false
            # if objective_value(subProb[s]) ≥ value.(mstr[:ζ])[s]
            push!(add_ray_restriction, s)
        
        elseif objective_value(subProbs[s]) ≥ value.(ζ)[s]
            canbreak = false
            push!(add_vertex_restriction, s)
        end
    end

    if canbreak
        println("======= Terminamos ======")
        break
    end
    
    println("======= SubProbs ======")
    for s ∈ add_ray_restriction
        @constraint(mstr, sum(dmnd[1:end, s].*value.(subProbs[s][:v])) - sum(avlb[j, s]*x[j] * sum(value.(subProbs[s][:w])[i, j] for i in 1:nI) for j in 1:nJ) ≤ 0)  # ¿Por que es un 0 vector en enunciado?
        println("Agregamos restricción de rayo en subproblema $s")
    end

    for s ∈ add_vertex_restriction
        @constraint(mstr, sum(dmnd[1:end, s].*value.(subProbs[s][:v])) - sum(avlb[j, s]*x[j] * sum(value.(subProbs[s][:w])[i, j] for i in 1:nI) for j in 1:nJ) ≤ ζ_val[s])  # ¿Por que es un 0 vector en enunciado?
        println("Agregamos restricción de vertice en subproblema $s")
    end

end

optimize!(mstr) 
println( "Valor óptimo: ", objective_value(mstr) )
println( "x = ", value.(x))

======= Iter 0 ======
======= Master ======


===== SubProbs =====


======= Iter 1 ======